In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%autoreload 1

In [6]:
%aimport model

In [7]:
import skorch

In [8]:
import numpy as np

In [9]:
import torch

In [10]:
import torchvision.datasets

In [11]:
torch.cuda.is_available()

True

In [10]:
!mkdir -p models

In [11]:
net = skorch.NeuralNetClassifier(
    module=model.WideResNet(
        "fashion-mnist",
        input_size=32,
        in_channels=1,
        classes=10,
        total_block_number=24,
        widen_factor=10,
    ),
    criterion=torch.nn.CrossEntropyLoss,
    batch_size=20,
    max_epochs=10,
    optimizer=lambda params, lr: torch.optim.SGD(params, lr=lr, momentum=0.9, nesterov=True),
    callbacks=[
        ('learning_rate_decay',
         skorch.callbacks.LRScheduler(
             policy=torch.optim.lr_scheduler.MultiStepLR,
             monitor=None,
             gamma=0.1,
             milestones=(150, 225)
         )
        ),
        ('progress_bar', skorch.callbacks.ProgressBar()),
        ('checkpoint', skorch.callbacks.Checkpoint(target="models/{net.module.name}.pt", monitor=None))
    ],
    device='cuda' if torch.cuda.is_available() else 'cpu',
    lr=0.1,
    train_split=skorch.dataset.CVSplit(5, stratified=False),
)
net.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=WideResNet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ResidualBlockGroup(
        (sequential): Sequential(
          (0): ResidualBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (dropout): Dropout(p=0.5)
            (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (conv_transform): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (1): ResidualBlock(
            (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1,

In [12]:
import RandomErasing.transforms

In [13]:
train_dataset = torchvision.datasets.FashionMNIST(
    "~/torchvision/datasets/FashionMNIST",
    download=True,
    train=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,)),
        RandomErasing.transforms.RandomErasing(mean=[0.4914]),
    ])
)

In [15]:
net.fit(train_dataset, None, epochs=300)

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=WideResNet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ResidualBlockGroup(
        (sequential): Sequential(
          (0): ResidualBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (dropout): Dropout(p=0.5)
            (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (conv_transform): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (1): ResidualBlock(
            (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1,

In [16]:
# net.fit_loop(train_dataset, None, epochs=300)

In [14]:
test_dataset = torchvision.datasets.FashionMNIST(
    "~/torchvision/datasets/FashionMNIST",
    download=True,
    train=False,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.Pad(2),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,)),
    ])
)